<a href="https://colab.research.google.com/github/kimdonggyu2008/audio_synthesis/blob/main/sound_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

import numpy as np




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sound_autoencoder import VAE

In [ ]:

LEARNING_RATE = 0.0005
BATCH_SIZE = 64
EPOCHS = 150

SPECTROGRAMS_PATH = ""#스펙트로그램 전처리 값들 위치


def load_fsdd(spectrograms_path): #스펙트로그램 읽어오기
    x_train = []
    for root, _, file_names in os.walk(spectrograms_path):#순차적으로 읽어옴
        for file_name in file_names:
            file_path = os.path.join(root, file_name)
            spectrogram = np.load(file_path) # (n_bins, n_frames, 1)
            x_train.append(spectrogram) #스펙트로그램 형태로 학습셋에 집어넣음
    x_train = np.array(x_train)# 해당 배열을 넘파이로 변환
    x_train = x_train[..., np.newaxis] # -> (3000, 256, 64, 1)# 넘파이 배열을 위해서 1차원 추가함
    return x_train


def train(x_train, learning_rate, batch_size, epochs): #학습
    autoencoder = VAE(
        input_shape=(256, 64, 1),
        conv_filters=(512, 256, 128, 64, 32), #프레임사이즈와 일치함, 반으로 계속 줄임
        conv_kernels=(3, 3, 3, 3, 3),
        conv_strides=(2, 2, 2, 2, (2, 1)),
        latent_space_dim=128
    )
    autoencoder.summary()
    autoencoder.compile(learning_rate)
    autoencoder.train(x_train, batch_size, epochs)
    return autoencoder


if __name__ == "__main__":
    x_train = load_fsdd(SPECTROGRAMS_PATH)
    autoencoder = train(x_train, LEARNING_RATE, BATCH_SIZE, EPOCHS)
    autoencoder.save("model")